In [1]:
cd ..

D:\J.H.LEE\05. CODING\Python\Project\tdatlib


In [10]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important;}</style>"))

import plotly.graph_objects as go
import plotly.offline as of
import pandas as pd
# import tdatlib as tdat
from plotly.subplots import make_subplots
from snob import tools, ecos, fred, krse, idex, kr

# Initialize

In [3]:
idex.period = ecos.period = fred.period = 20

tickers = pd.DataFrame(
    data=[
        dict(ticker="5046",   종류="지수", 이름="KRX은행"),
        dict(ticker="091170", 종류="ETF", 이름="KODEX은행"),
        dict(ticker="091220", 종류="ETF", 이름="TIGER은행"),
        dict(ticker="105560", 종류="주식", 이름="KB금융"),
        dict(ticker="086790", 종류="주식", 이름="하나금융지주"),
        dict(ticker="139130", 종류="주식", 이름="DGB금융지주"),
        dict(ticker="055550", 종류="주식", 이름="신한지주"),
        dict(ticker="024110", 종류="주식", 이름="기업은행"),
        dict(ticker="138930", 종류="주식", 이름="BNK금융지주"),
        dict(ticker="316140", 종류="주식", 이름="우리금융지주"),
        dict(ticker="038540", 종류="주식", 이름="상상인"),
    ]
).set_index(keys='ticker').join(other=krse.overview)

stock_objs = {ticker: kr(ticker=ticker) for ticker in tickers[tickers.종류 == '주식'].index}

In [4]:
# 레이아웃
layout = dict(
    plot_bgcolor='white', height=750,
    legend=dict(orientation="h", xanchor="right", yanchor="bottom", x=0.96, y=1),
    xaxis_rangeslider=dict(visible=False)
)

# Data Collection

## KRX은행지수

In [ ]:
# KRX 은행 지수
krx_bank = idex.bank
krx_bank_trace = tools.draw_line(krx_bank.종가, 'KRX은행')
krx_bank_trace.color = 'black'
krx_bank_trace.width = 1.0

## 수신/대출 금리

### 금리 정보

In [12]:
col_table = pd.concat(
    objs={
        "수신신규 121Y002":ecos.contains('121Y002').이름.drop_duplicates(keep='first'),
        "수신잔액 121Y013":ecos.contains('121Y013').이름.drop_duplicates(keep='first'),
        "대출신규 121Y006":ecos.contains('121Y006').이름.drop_duplicates(keep='first'),
        "대출잔액 121Y015":ecos.contains('121Y015').이름.drop_duplicates(keep='first'),
    }, axis=1
)
col_table

,수신신규 121Y002,수신잔액 121Y013,대출신규 121Y006,대출잔액 121Y015
0,저축성수신,총수신(요구불예금 및 수시입출식 저축성예금 포함),대출평균 1),총대출 1)
3,순수저축성예금 1),수시입출식 저축성예금,기업대출,총대출(당좌대출 제외)
6,정기예금,저축성수신(요구불예금 및 수시입출식 저축성예금 제외),대기업대출,기업대출 2)
9,정기예금(6개월 미만) 2),저축성수신(금융채 제외),중소기업대출,대기업대출
12,정기예금(6개월~1년 미만) 2),순수저축성예금,운전자금대출,중소기업대출
15,정기예금(1~2년 미만) 2),정기예금,시설자금대출,운전자금대출
18,정기예금(2~3년 미만),정기적금,가계대출,시설자금대출
21,정기예금(3~4년 미만),상호부금,소액대출 (500만원 이하),가계대출 2)
24,정기예금(4~5년 미만),주택부금,주택담보대출,소액대출 (500만원 이하)
27,정기예금(5년 이상),시장형금융상품,예·적금담보대출,주택담보대출


### 금리 데이터

In [14]:
depo = ecos.load('121Y002', '저축성수신(금융채 제외) 1)')

# loan = ecos.load('121Y015', '총대출 1)')
loan = ecos.load('121Y006', '가계대출')
d_depo_loan = loan - depo

depo_trace = tools.draw_line(depo, name=depo_col)
loan_trace = tools.draw_line(loan, name=loan_col)
d_depo_loan_trace = tools.draw_line(d_depo_loan, name='예대출금리차')

## 기준 금리

In [6]:
# 한국은행 기준금리
kr_std = ecos.기준금리
kr_std_trace = tools.draw_line(kr_std, name='한국 기준금리')
kr_std_trace.yhoverformat = '.2f%'
kr_std_trace.line = dict(width=1.2)

# 미국 기준금리
us_std = fred.기준금리
us_std_trace = tools.draw_line(us_std, name='미국 기준금리')
us_std_trace.yhoverformat = '.2f%'
us_std_trace.line = dict(width=1.2)

## 3년물 금리

In [7]:
# 한국채 3년물 금리
kr_3yt = ecos.국고채3Y
kr_3yt_trace = tools.draw_line(kr_3yt, name='한국채3년물')
kr_3yt_trace.line = dict(width=1.2)
kr_3yt_trace.yhoverformat = '.2f%'


# 미국채 3년물 금리
us_3yt = fred.국고채3Y
us_3yt_trace = tools.draw_line(us_3yt, name='미국채3년물')
us_3yt_trace.line = dict(width=1.2)
us_3yt_trace.yhoverformat ='.2f%'

## 10년물 금리

In [8]:
# 한국채 10년물 금리
kr_10yt = ecos.국고채10Y
kr_10yt_trace = tools.draw_line(kr_10yt, name='한국채10년물')
kr_10yt_trace.line = dict(width=1.2)
kr_10yt_trace.yhoverformat = '.2f%'


# 미국채 10년물 금리
us_10yt = fred.국고채10Y
us_10yt_trace = tools.draw_line(us_10yt, name='미국채10년물')
us_10yt_trace.line = dict(width=1.2)
us_10yt_trace.yhoverformat = '.2f%'

## 원-달러 환율

In [ ]:
# 원-달러 환율
exchange = ecos.원달러환율
exchange_trace = tools.draw_candle(exchange, name='원달러환율')

# BM & Products

- 은행 사업 구조 및 최대 매출처 확인
- 주요 매출처
  * 유가 증권 평가 및 처분 이익<br>
    : 1년 또는 1영업주기 내 현금화할 의도가 있는 증권을 유가 증권이라고 한다.<br>
    : 증권은 CD, CP 등 단기 증서 및 보통주/우선주의 주식과 사채, 어음 등 채무 증권을 말한다.<br>
    : 이러한 유가 증권을 취득 가액보다 높은 이익으로 처분할 때 발생하는 이익 및 보유 증권의 평가액을 매출에 반영한다.<br>
  * 신탁 업무 운용 수익
    : 운용

## 대표 시중은행 매출 구조

In [18]:
target = tickers.sort_values(by='시가총액', ascending=False)[:4]
fig = make_subplots(
    rows=2, cols=2,
    vertical_spacing=0.12, horizontal_spacing=0.1,
    subplot_titles=tuple(target.이름.tolist()),
    specs=[
        [{"type": "domain"}, {"type": "domain"}],
        [{"type": "domain"}, {"type": "domain"}]
    ]
)

for n, ticker in enumerate(target.index):
    obj = stock_objs[ticker]
    products = obj.fnguide.RecentProducts
    fig.add_trace(
        trace = go.Pie(
            name='Product',
            labels=products.index,
            values=products,
            visible=True,
            showlegend=True,
            automargin=True,
            textfont=dict(color='white'),
            textinfo='label+percent',
            insidetextorientation='radial',
            hoverinfo='label+percent',
        ), row = n // 2 + 1, col = n % 2 + 1
    )

fig.update_layout(layout)
fig.update_layout(    
    title=f'<b>은행주</b> 주요 매출',
    legend=dict(y=1.08)
)

for n, annotation in enumerate(fig['layout']['annotations']):
    annotation['x'] = 0 + 0.55 * (n % 2)
    annotation['xanchor'] = 'center'
    annotation['xref'] = 'paper'
fig.show()

In [24]:
fig = make_subplots(
    rows=2, cols=2,
    vertical_spacing=0.12, horizontal_spacing=0.1,
    subplot_titles=tuple(target.이름.tolist()),
    specs=[
        [{"type": "bar"}, {"type": "bar"}],
        [{"type": "bar"}, {"type": "bar"}]
    ]
)

for n, ticker in enumerate(target.index):
    obj = stock_objs[ticker]
    products = obj.fnguide.AllProducts
    for col in products.columns[::-1]:
        fig.add_trace(
            trace=go.Bar(
                x=products.index, y=products[col], name=col, 
                legendgroup=col, showlegend=True if not n else False,
                text=products[col], textposition="inside", texttemplate=col + "<br>%{y:.2f}%", hoverinfo='skip',                
            ), row = n // 2 + 1, col = n % 2 + 1
        )
fig.update_layout(layout, barmode="stack")
fig.update_layout(    
    title=f'<b>은행주</b> 연도별 주요 매출',
    legend=dict(y=1.08)
)

## 매출 비중

In [ ]:
p = pd.concat(objs=[stock_objs[ticker].fnguide.Products for n, ticker in enumerate(target.index)], axis=1).T.sum()
portion = round(100 * p / p.sum(), 2).sort_values()

fig = go.Figure()
fig.add_trace(
    trace=go.Bar(
        x=portion, y=portion.index, orientation='h',
        texttemplate='%{x:.2f}%', textposition='inside', hoverinfo='skip'
    )
)
fig.update_layout(
    title='시중은행 주요 매출처 비중', plot_bgcolor='white', height=500,
    xaxis=dict(title='비중[%]', showgrid=True, gridcolor='lightgrey', showline=False, zeroline=False),
    yaxis=dict(showgrid=False, zeroline=False),
)

# Macro-Relation

## KRX은행 및 예대금리차

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(trace=krx_bank_trace(), secondary_y=False)
fig.add_trace(trace=depo_trace(), secondary_y=True)
fig.add_trace(trace=loan_trace(), secondary_y=True)
fig.add_trace(trace=d_depo_loan_trace(), secondary_y=True)
# fig.add_trace(trace=kr_std_trace(), secondary_y=True)
# fig.add_trace(trace=us_std_trace(), secondary_y=True)
# fig.add_trace(trace=kr_2yt_trace(), secondary_y=True)
# fig.add_trace(trace=us_2yt_trace(), secondary_y=True)
# fig.add_trace(trace=us_10yt_trace(), secondary_y=True)

tools.draw_recession(fig, market='kr')
fig.update_layout(layout)
fig.update_layout(
    title=f'<b>KRX은행지수</b>와 금리',
    xaxis=dict(
        title='날짜',
        tickformat='%Y/%m/%d',
        showticklabels=True,
        showgrid=True,
        gridcolor='lightgrey'
    ),
    yaxis=dict(
        title='지수',
        showticklabels=True,
        showgrid=True,
        gridcolor='lightgrey'
    ),
    yaxis2=dict(
        title='이율[%]',
        showticklabels=True
    )
)

## KRX은행 및 원-달러 환율

In [ ]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(trace=krx_bank_trace(), secondary_y=False)
fig.add_trace(trace=exchange_trace(), secondary_y=True)
tdat.draw_recession(fig, market='kr')
fig.update_layout(layout)
fig.update_layout(
    title=f'<b>KRX은행지수</b>와 원-달러 환율',
    xaxis=dict(
        title='날짜',
        tickformat='%Y/%m/%d',
        showticklabels=True,
        showgrid=True,
        gridcolor='lightgrey',
    ),
    yaxis=dict(
        title='지수',
        showticklabels=True,
        showgrid=True,
        gridcolor='lightgrey'
    ),
    yaxis2=dict(
        title='KRW',
        showticklabels=True
    ),
    xaxis_rangeslider=dict(visible=False)
)

# Foreigners

In [ ]:
fig = make_subplots(
    rows=2, cols=2,
    vertical_spacing=0.12, horizontal_spacing=0.1,
    subplot_titles=tuple(target.이름.tolist()),
    specs=[
        [{"type": "xy", "secondary_y": True}, {"type": "xy", "secondary_y": True}],
        [{"type": "xy", "secondary_y": True}, {"type": "xy", "secondary_y": True}]
    ]
)

for n, ticker in enumerate(target.index):
    obj = stock_objs[ticker]
    foreigner = obj.fnguide.Foreigner['3Y'].dropna()
    fig.add_trace(
        trace = go.Scatter(
            name=ticker,
            x=foreigner.index, y=foreigner['외국인보유비중'],
            showlegend=False,
            xhoverformat='%Y/%m/%d', 
            hovertemplate='%{x}<br>%{y:.2f}%<extra>보유비중</extra>'
        ), row = n // 2 + 1, col = n % 2 + 1, secondary_y=False
    )
    fig.add_trace(
        trace = go.Scatter(
            name=ticker,
            x=foreigner.index, y=foreigner['종가'],
            line=dict(color='#000000', dash='dot', width=1),
            showlegend=False,
            xhoverformat='%Y/%m/%d', 
            hovertemplate='%{x}<br>%{y:,d}원<extra>종가</extra>'
        ), row = n // 2 + 1, col = n % 2 + 1, secondary_y=True
    )
    

tdat.draw_recession(fig, market='kr')
fig.update_layout(layout)
fig.update_layout(    
    title=f'<b>은행주</b> 외국인 보유비중',
    xaxis=dict(showgrid=True, gridcolor='lightgrey', tickformat="%Y/%m/%d"),
    xaxis2=dict(showgrid=True, gridcolor='lightgrey', tickformat="%Y/%m/%d"),
    xaxis3=dict(showgrid=True, gridcolor='lightgrey', tickformat="%Y/%m/%d"),
    xaxis4=dict(showgrid=True, gridcolor='lightgrey', tickformat="%Y/%m/%d"),
    yaxis=dict(title='[%]', showgrid=True, gridcolor='lightgrey'),
    yaxis2=dict(title='[KRW]', showgrid=False),
    yaxis3=dict(title='[%]', showgrid=True, gridcolor='lightgrey'),
    yaxis4=dict(title='[KRW]', showgrid=False),
    yaxis5=dict(title='[%]', showgrid=True, gridcolor='lightgrey'),
    yaxis6=dict(title='[KRW]', showgrid=False),
    yaxis7=dict(title='[%]', showgrid=True, gridcolor='lightgrey'),
    yaxis8=dict(title='[KRW]', showgrid=False),
)

for n, annotation in enumerate(fig['layout']['annotations']):
    annotation['x'] = 0 + 0.55 * (n % 2)
    annotation['xanchor'] = 'center'
    annotation['xref'] = 'paper'
fig.show()

# 습작

In [15]:
ticker = '316140'

stock = kr(ticker=ticker)
data = stock.fnguide.AllProducts
data

,유가증권평가및처분이익,이자수익,수수료수익,외환거래이익,기타(계)
기말,,,,,
2019/12,41.49,46.58,7.53,2.65,1.75
2020/12,56.93,33.26,5.92,2.65,1.24
2021/12,50.70,36.39,7.19,2.07,3.65


In [15]:
fig = go.Figure()

fig.add_trace(kr_std_trace)
fig.add_trace(kr_3yt_trace)
fig.add_trace(kr_10yt_trace)
fig.add_trace(loan_trace)

# fig.show()
of.plot(fig, filename='file.html')

'file.html'